In [1]:
import gzip
from collections import defaultdict
import random
import numpy
import scipy.optimize

In [2]:
path = 'amazon_reviews_us_Musical_Instruments_v1_00.tsv.gz'

In [3]:
f = gzip.open(path, 'rt', encoding='utf8')

In [4]:
header = f.readline()
header = header.strip().split('\t')

In [5]:
dataset = []

In [6]:
for line in f:
    fields = line.strip().split('\t')
    d = dict(zip(header, fields))
    d['star_rating'] = int(d['star_rating'])
    d['helpful_votes'] = int(d['helpful_votes'])
    d['total_votes'] = int(d['total_votes'])
    dataset.append(d)

EOFError: Compressed file ended before the end-of-stream marker was reached

In [7]:
dataset[0]

{'marketplace': 'US',
 'customer_id': '45610553',
 'review_id': 'RMDCHWD0Y5OZ9',
 'product_id': 'B00HH62VB6',
 'product_parent': '618218723',
 'product_title': 'AGPtek® 10 Isolated Output 9V 12V 18V Guitar Pedal Board Power Supply Effect Pedals with Isolated Short Cricuit / Overcurrent Protection',
 'product_category': 'Musical Instruments',
 'star_rating': 3,
 'helpful_votes': 0,
 'total_votes': 1,
 'vine': 'N',
 'verified_purchase': 'N',
 'review_headline': 'Three Stars',
 'review_body': 'Works very good, but induces ALOT of noise.',
 'review_date': '2015-08-31'}

In [8]:
# useful data structures

In [9]:
usersPerItem = defaultdict(set)
itemsPerUser = defaultdict(set)

In [10]:
itemNames = {}

In [11]:
for d in dataset:
    user,item = d['customer_id'], d['product_id']
    usersPerItem[item].add(user)
    itemsPerUser[user].add(item)
    itemNames[item] = d['product_title']

In [12]:
def Jaccard(s1, s2):
    numer = len(s1.intersection(s2))
    denom = len(s1.union(s2))
    return numer / denom

In [14]:
def mostSimilar(i):
    similarities = []
    users = usersPerItem[i]
    for i2 in usersPerItem:
        if i2 == i: continue
        sim = Jaccard(users, usersPerItem[i2])
        similarities.append((sim,i2))
    similarities.sort(reverse=True)
    return similarities[:10]

In [15]:
dataset[2]

{'marketplace': 'US',
 'customer_id': '6111003',
 'review_id': 'RIZR67JKUDBI0',
 'product_id': 'B0006VMBHI',
 'product_parent': '603261968',
 'product_title': 'AudioQuest LP record clean brush',
 'product_category': 'Musical Instruments',
 'star_rating': 3,
 'helpful_votes': 0,
 'total_votes': 1,
 'vine': 'N',
 'verified_purchase': 'Y',
 'review_headline': 'Three Stars',
 'review_body': 'removes dust. does not clean',
 'review_date': '2015-08-31'}

In [16]:
query = dataset[2]['product_id']

In [18]:
# top 10 most similar items to the query item using the jaccard similarity
mostSimilar(query)

[(0.023121387283236993, 'B00006I5SD'),
 (0.019230769230769232, 'B00E7MVP3S'),
 (0.013422818791946308, 'B00HZRXBUC'),
 (0.013157894736842105, 'B00008BWM7'),
 (0.012269938650306749, 'B000WMCEKK'),
 (0.012096774193548387, 'B00006I5SB'),
 (0.010638297872340425, 'B000AJR482'),
 (0.007751937984496124, 'B012OFHBJ6'),
 (0.007751937984496124, 'B00NFSGCRO'),
 (0.007751937984496124, 'B00K5927TS')]

In [19]:
itemNames[query]

'AudioQuest LP record clean brush'

In [20]:
[itemNames[x[1]] for x in mostSimilar(query)]

['Shure SFG-2 Stylus Tracking Force Gauge',
 'Signstek Blue LCD Backlight Digital Long-Playing LP Turntable Stylus Force Scale Gauge Tester',
 'Record Doctor - Stylus Cleaning Fluid with Brush',
 'Shure M97xE High-Performance Magnetic Phono Cartridge',
 'Ortofon - 2M Red MM Phono Cartridge',
 'Shure M97xE High-Performance Magnetic Phono Cartridge',
 'ART Pro Audio DJPRE II Phono Turntable Preamplifier',
 'LyxPro LDC-20 Large Diaphragm Cardioid Condenser Studio Microphone, Shockmount, Foam Windscreen & Case',
 'XLR Microphone Cable 30ft',
 'Ortofon Stylus Guards']

In [22]:
# more eficient implementation
def mostSimilarFast(i):
    similarities = []
    users = usersPerItem[i]
    candidateItems = set()
    for u in users:
        candidateItems = candidateItems.union(itemsPerUser[u])
    for i2 in candidateItems:
        if i2 == i: continue
        sim = Jaccard(users, usersPerItem[i2])
        similarities.append((sim,i2))
    similarities.sort(reverse=True)
    return similarities[:10]

In [23]:
mostSimilarFast(query)

[(0.023121387283236993, 'B00006I5SD'),
 (0.019230769230769232, 'B00E7MVP3S'),
 (0.013422818791946308, 'B00HZRXBUC'),
 (0.013157894736842105, 'B00008BWM7'),
 (0.012269938650306749, 'B000WMCEKK'),
 (0.012096774193548387, 'B00006I5SB'),
 (0.010638297872340425, 'B000AJR482'),
 (0.007751937984496124, 'B012OFHBJ6'),
 (0.007751937984496124, 'B00NFSGCRO'),
 (0.007751937984496124, 'B00K5927TS')]

Similarity-based recommender for rating prediction

In [24]:
# more utility data structures

In [27]:
reviewsPerUser = defaultdict(list)
reviewsPerItem = defaultdict(list)

In [28]:
for d in dataset:
    user,item = d['customer_id'], d['product_id']
    reviewsPerUser[user].append(d)
    reviewsPerItem[item].append(d)

In [29]:
ratingMean = sum([d['star_rating'] for d in dataset]) / len(dataset)

In [30]:
ratingMean

4.2936885727886205

In [35]:
def predictRating(user, item):
    ratings = []
    similarities = []
    for d in reviewsPerUser[user]:
        i2 = d['product_id']
        if i2 == item: continue
        ratings.append(d['star_rating'])
        similarities.append(Jaccard(usersPerItem[item], usersPerItem[i2]))
    if (sum(similarities) > 0):
        weightedRatings = [(x*y) for x,y in zip(ratings,similarities)]
        return sum(weightedRatings) / sum(similarities)
    else:
        # User hasn't rated any similar items
        return ratingMean

In [36]:
dataset[1]

{'marketplace': 'US',
 'customer_id': '14640079',
 'review_id': 'RZSL0BALIYUNU',
 'product_id': 'B003LRN53I',
 'product_parent': '986692292',
 'product_title': 'Sennheiser HD203 Closed-Back DJ Headphones',
 'product_category': 'Musical Instruments',
 'star_rating': 5,
 'helpful_votes': 0,
 'total_votes': 0,
 'vine': 'N',
 'verified_purchase': 'Y',
 'review_headline': 'Five Stars',
 'review_body': 'Nice headphones at a reasonable price.',
 'review_date': '2015-08-31'}

In [37]:
u,i = dataset[1]['customer_id'], dataset[1]['product_id']

In [38]:
predictRating(u,i)

5.0

In [44]:
def MSE(predictions, labels):
    differences = [(x-y)**2 for x,y in zip(predictions, labels)]
    return sum(differences) / len(differences)

In [45]:
alwasyPredictMean = [ratingMean for d in dataset]

In [46]:
cfPredictions = [predictRating(d['customer_id'], d['product_id']) for d in dataset]

In [48]:
labels = [d['star_rating'] for d in dataset]

In [49]:
MSE(alwasyPredictMean, labels)

1.45376606289067

In [50]:
MSE(cfPredictions, labels)

1.5639964910086366

Implementing Latent Factor Models

In [52]:
N = len(dataset)
nUsers = len(reviewsPerUser)
nItems = len(reviewsPerItem)
users = list(reviewsPerUser.keys())
items = list(reviewsPerItem.keys())

In [53]:
alpha = ratingMean

In [54]:
userBiases = defaultdict(float)
itemBiases = defaultdict(float)

In [55]:
def prediction(user, item):
    return alpha + userBiases[user] + itemBiases[item]

In [56]:
def unpack(theta):
    global alpha
    global userBiases
    global itemBiases
    alpha = theta[0]
    userBiases = dict(zip(users, theta[1:nUsers+1]))
    itemBiases = dict(zip(items, theta[1+nUsers:]))

In [64]:
def cost(theta, lebals, lamb):
    unpack(theta)
    predictions = [prediction(d['customer_id'], d['product_id']) for d in dataset]
    cost = MSE(predictions, lebals)
    print('MSE = ' + str(cost))
    for u in userBiases:
        cost += lamb*userBiases[u]**2
    for i in itemBiases:
        cost += lamb*itemBiases[i]**2
    return cost

In [65]:
def derivative(theta, labels, lamb):
    unpack(theta)
    N = len(dataset)
    dalpha = 0
    dUserBiases = defaultdict(float)
    dItemBiases = defaultdict(float)
    for d in dataset:
        u,i = d['customer_id'], d['product_id']
        pred = prediction(u,i)
        diff = pred - d['star_rating']
        dalpha += 2/N*diff
        dUserBiases[u] += 2/N*diff
        dItemBiases[i] + 2/N*diff
    for u in userBiases:
        dUserBiases[u] += 2*lamb*userBiases[u]
    for i in itemBiases:
        dItemBiases[i] += 2*lamb*itemBiases[i]
    dtheta = [dalpha] + [dUserBiases[u] for u in users] + [dItemBiases[i] for i in items]
    return numpy.array(dtheta)

In [66]:
MSE(alwasyPredictMean, labels)

1.45376606289067

In [67]:
scipy.optimize.fmin_l_bfgs_b(cost, [ratingMean] + [0.0]*(nUsers+nItems), derivative, args = (labels, 0.001))

MSE = 1.45376606289067
MSE = 1.4490282440855686
MSE = 2.3942350837247575
MSE = 1.4490062673724735
MSE = 1.4427695375640965
MSE = 1.4427626207944249
MSE = 1.442742754155659


(array([ 4.29244060e+00, -3.60850409e-03,  1.36046357e-02, ...,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00]),
 1.4482080792982428,
 {'grad': array([-5.34095002e-10,  7.11388531e-10,  5.26528005e-09, ...,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00]),
  'task': b'CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL',
  'funcalls': 7,
  'nit': 5,
  'warnflag': 0})

Optimised latent factor model

In [87]:
alpha = ratingMean
userBias = defaultdict(float)
itemBiases = defaultdict(float)
userGamma = {}
itemGamma = {}

K = 2  #  number of laten factors (i.e. dimensionality of gamma)

In [88]:
for u in reviewsPerUser:
    userGamma[u] = [random.random() * 0.1 - 0.05 for k in range(K)]

In [89]:
for i in reviewsPerItem:
    itemGamma[i] = [random.random() * 0.1 - 0.05 for k in range(K)]

In [97]:
def unpack(theta):
    global alpha
    global userBiases
    global itemBiases
    global userGamma
    global itemGamma
    index = 0
    alpha = theta[index]
    index += 1
    userBiases = dict(zip(users, theta[index:index+nUsers]))
    index += nUsers
    itemBiases = dict(zip(items, theta[index:index+nItems]))
    index += nItems
    for u in users:
        userGamma[u] = theta[index:index+K]
        index += K
    for i in items:
        itemGamma[i] = theta[index:index+K]
        index += K

In [98]:
def inner(x, y):
    return sum([a*b for a,b in zip(x,y)])

In [99]:
def prediction(user, item):
    return alpha + userBiases[user] + itemBiases[item] + inner(userGamma[user], itemGamma[item])

In [100]:
def cost(theta, labels, lamb):
    unpack(theta)
    predictions = [prediction(d['customer_id'], d['product_id']) for d in dataset]
    cost = MSE(predictions, labels)
    print("MSE = " + str(cost))
    for u in users:
        cost += lamb*userBiases[u]**2
        for k in range(K):
            cost += lamb*userGamma[u][k]**2
    for i in items:
        cost += lamb*itemBiases[i]**2
        for k in range(K):
            cost += lamb*itemGamma[i][k]**2
    return cost

In [101]:
def derivative(theta, labels, lamb):
    unpack(theta)
    N = len(dataset)
    dalpha = 0
    dUserBiases = defaultdict(float)
    dItemBiases = defaultdict(float)
    dUserGamma = {}
    dItemGamma = {}
    for u in reviewsPerUser:
        dUserGamma[u] = [0.0 for k in range(K)]
    for i in reviewsPerItem:
        dItemGamma[i] = [0.0 for k in range(K)]
    for d in dataset:
        u,i = d['customer_id'], d['product_id']
        pred = prediction(u, i)
        diff = pred - d['star_rating']
        dalpha += 2/N*diff
        dUserBiases[u] += 2/N*diff
        dItemBiases[i] += 2/N*diff
        for k in range(K):
            dUserGamma[u][k] += 2/N*itemGamma[i][k]*diff
            dItemGamma[i][k] += 2/N*userGamma[u][k]*diff
    for u in userBiases:
        dUserBiases[u] += 2*lamb*userBiases[u]
        for k in range(K):
            dUserGamma[u][k] += 2*lamb*userGamma[u][k]
    for i in itemBiases:
        dItemBiases[i] += 2*lamb*itemBiases[i]
        for k in range(K):
            dItemGamma[i][k] += 2*lamb*itemGamma[i][k]
    dtheta = [dalpha] + [dUserBiases[u] for u in users] + [dItemBiases[i] for i in items]
    for u in users:
        dtheta += dUserGamma[u]
    for i in items:
        dtheta += dItemGamma[i]
    return numpy.array(dtheta)

In [104]:
scipy.optimize.fmin_l_bfgs_b(cost, [ratingMean] + [0.0]*(nUsers*(K+1)+nItems*(K+1)), derivative, args = (labels, 0.001))

MSE = 1.45376606289067
MSE = 1.4407372319287919
MSE = 3.891352286921449
MSE = 1.4403721638021079
MSE = 1.4152985807396175
MSE = 1.412382743659729
MSE = 1.4018587445961355
MSE = 1.400926563519946
MSE = 1.4005203203485421
MSE = 1.400548172478613
MSE = 1.400533532871454
MSE = 1.4005605767302904


(array([ 4.28927451e+00, -3.59590651e-03,  1.34693885e-02, ...,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00]),
 1.4230193813962195,
 {'grad': array([-5.39722873e-08,  1.30579856e-08, -2.18067538e-08, ...,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00]),
  'task': b'CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL',
  'funcalls': 12,
  'nit': 9,
  'warnflag': 0})